# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [25]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass(slots=True) #improved
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [26]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [27]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

Let's take a look at the task as a computational graph (what it actually is) ~~say 'hello' to TensorFlow~~. Code beneath covers tasks 1.1, 1.2 and 1.5

In [28]:
import numpy as np
import sympy
import random
from abc import ABC, abstractmethod

In [29]:
np.random.seed(2023)
random.seed(2023)

In [30]:
class Node(ABC):
    """Node of computational graph. Abstract class."""    
    @abstractmethod
    def backward(self, var):
        """Create new node in graph.
        """        
        pass
    
    @abstractmethod
    def compute(self):
        """TBD."""        
        pass

In [31]:
class Const(Node):
    """Representates const type of node.
    """    
    def __init__(self, value: float | int):
        self.value = value
    
    def backward(self, var):
        return Const(0.0)

    def compute(self):
        return self.value
    
    def __repr__(self):
        return str(self.value)

In [32]:
class Variable(Node):
    """Representation of variable.
    """    
    def __init__(self, name, value=None):
        self.name = name
        self.value = value
    
    def backward(self, var):
        return Const(1) if self == var else Const(0)

    def compute(self):
        if self.value is None:
            raise ValueError('variable seems to be empty')
        return self.value

    def __repr__(self):
        return f'{self.name}'

In [33]:
class Sum(Node):
    """Summary operation.
    """
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def backward(self, var):
        return Sum(self.x.backward(var), self.y.backward(var))

    def compute(self):
        return self.x.compute() + self.y.compute()
    
    def __repr__(self):
        return f'({self.x} + {self.y})'

In [34]:
class Mul(Node):
    """Multiply operation.
    """
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def backward(self, var):
        return Sum(
            Mul(self.x.backward(var), self.y),
            Mul(self.x, self.y.backward(var))
        )

    def compute(self):
        return self.x.compute() * self.y.compute()
    
    def __repr__(self):
        return f'({self.x} * {self.y})'

In [35]:
class Neg(Node):
    """Change sign of Node to opposite.
    x > 0 -> Neg(x) -> x < 0
    """
    def __init__(self, x):
        self.x =  x
    
    def backward(self, var):
        return Neg(self.x.backward(var))
    
    def compute(self):
        return - self.x.compute()

    def __repr__(self):
        return f'(-{self.x})'

In [36]:
class Power(Node):
    """Power operation.
    Notice, that first argument raises to power from second argument.
    """
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def backward(self, var):
        return Sum(
            Mul(Mul(self.y, Power(self.x, Sum(self.y, Const(-1)))), self.x.backward(var)),
            Mul(Mul(Power(self.x, self.y), self.y.backward(var)), Log(self.x))
        )
    
    def compute(self):
        return self.x.compute() ** self.y.compute()
    
    def __repr__(self):
        return f'({self.x} ** {self.y})'

In [37]:
class Divide(Node):
    """Divide first argument to the second.
    """
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def backward(self, var):
        return Divide(
            Sum(
                Mul(self.x.backward(var), self.y),
                Neg(Mul(self.y.backward(var), self.x))
            ),
            Power(self.y, Const(2))
        )
    
    def compute(self):
        return self.x.compute() / self.y.compute()
    
    def __repr__(self):
        return f'({self.x} / {self.y})'

In [38]:
class Abs(Node):
    """Represent absolute operation. 
    Notice that abs(x) -- not smooth-function, so it may have some unexpected side-effects 🤓.\n
    This implementaion is based on assumption that if submodule expression is lower than zero,
    then it's diff and value change their signs. Otherwise nothing happens.
    """
    def __init__(self, x):
        self.x =  x
    
    def backward(self, var):
        return Abs(self.x.backward(var))
    
    def compute(self):
        value = self.x.compute()
        if value == 0:
            return 0
        return value if value > 0 else -value
    
    def __repr__(self):
        return f'(|{self.x}|)'

In [39]:
x = Variable('x', 3)
y = Variable('y', 2)

z = Abs(Sum(y, Neg(x)))

z


(|(y + (-x))|)

In [40]:
z.backward(x)

(|(0 + (-1))|)

In [41]:
z.backward(x).compute()

1

In [53]:
#x * y + z - 5 * y  
x = Variable('x', 10)
y = Variable('y', 10)
z = Variable('z', 10)

f = Sum(
    Sum(
        Mul(x,y),
        z 
    ), Neg(Mul(Const(5),y))
)
f

(((x * y) + z) + (-(5 * y)))

In [54]:
f.backward(x).compute()


10.0

In [55]:
x = Variable('x', 3)
y = Variable('y', 2)
z = Power(x,y)
z

(x ** y)

In [56]:
z.compute()

9

In [57]:
z.backward(y)

(((y * (x ** (y + -1))) * 0) + (((x ** y) * 1) * log(x)))

In [58]:
z.backward(y).compute()

9.887510598012987

Let's make our toy auto diff easier to use: add symbolic operations. To not-built-in functions add our mixin to class directly:

In [59]:
from typing import Any
class AutoGradMixin:
    """Mixin for covering all nodes with symbolic operators.
    """    
    @staticmethod
    def _to_node(x: Node | Any):
        """check if 'x' is Node, otherwise create Const object base on 'x'

        Args:
            x (Node | Any): Node or basement for a new Const
        """
        return x if isinstance(x, Node) else Const(x) 

    def __add__(self, other):
        return SymbolicSum(self, self._to_node(other))

    def __radd__(self, other):
        return SymbolicSum(self._to_node(other), self)

    def __mul__(self, other):
        return SymbolicMul(self, self._to_node(other))     

    def __rmul__(self, other):
        return SymbolicMul(self._to_node(other), self)   

    def __neg__(self):
        return SymbolicNeg(self)
    
    def __sub__(self, other):
        return SymbolicSum(self, SymbolicNeg(self._to_node(other)))
    
    def __rsub__(self, other):
        return SymbolicSum(self._to_node(other), SymbolicNeg(self))
    
    def __pow__(self, other):
        return SymbolicPower(self, self._to_node(other))
    
    def __rpow__(self, other):
        return SymbolicPower(self._to_node(other), self)
    
    def __truediv__(self, other):
        return SymbolicDivide(self, self._to_node(other))
    
    def __rtruediv__(self, other):
        return SymbolicDivide(self._to_node(other), self)
    
    def __abs__(self):
        return Abs(self)

class SymbolicVar(Variable, AutoGradMixin):
    pass

class SymbolicConst(Const, AutoGradMixin):
    pass

class SymbolicSum(Sum, AutoGradMixin):
    pass

class SymbolicMul(Mul, AutoGradMixin):
    pass

class SymbolicNeg(Neg, AutoGradMixin):
    pass

class SymbolicPower(Power, AutoGradMixin):
    pass

class SymbolicDivide(Divide, AutoGradMixin):
    pass

class Abs(Node, AutoGradMixin):
    """Represent absolute operation. 
    Notice that abs(x) -- not smooth-function, so it may have some unexpected side-effects 🤓.\n
    This implementaion is based on assumption that if submodule expression is lower than zero,
    then it's diff and value change their signs. Otherwise nothing happens.
    """
    def __init__(self, x):
        self.x =  x
    
    def backward(self, var):
        return Abs(self.x.backward(var))
    
    def compute(self):
        value = self.x.compute()
        if value == 0:
            return 0
        return value if value > 0 else -value
    
    def __repr__(self):
        return f'(|{self.x}|)'

class Log(Node, AutoGradMixin):
    """Represent natural logarithm.
    """
    def __init__(self, x) -> None:
        self.x = x
    
    def backward(self, var):
        return Mul(
            Divide(Const(1), self.x),
            self.x.backward(var)
            )
    
    def compute(self):
        return np.log(self.x.compute())
    
    def __repr__(self):
        return f'log({self.x})'

class Exp(Node, AutoGradMixin):
    """Represent exponential operation
    """
    def __init__(self, x) -> None:
        self.x = x
    
    def backward(self, var):
        return Mul(Exp(self.x), self.x.backward(var))
    
    def compute(self):
        return np.exp(self.x.compute())
    
    def __repr__(self):
        return f'e^({self.x})'

class Cos(Node, AutoGradMixin):
    """Represent cosine operation
    """
    def __init__(self, x) -> None:
        self.x = x
    
    def backward(self, var):
        return Mul(Neg(Sin(self.x)), self.x.backward(var))
    
    def compute(self):
        return np.cos(self.x.compute())
    
    def __repr__(self):
        return f'cos({self.x})'

class Sin(Node, AutoGradMixin):
    """Represent sinus operation
    """
    def __init__(self, x) -> None:
        self.x = x
    
    def backward(self, var):
        return Mul(Cos(self.x), self.x.backward(var))
    
    def compute(self):
        return np.sin(self.x.compute())
    
    def __repr__(self):
        return f'sin({self.x})'

Testing time! 🛠

Let's use just simple "asserts" with sympy for this moment

In [60]:
#testing binary ops, division and powering
x_value, y_value = np.random.randint(1,100), np.random.randint(1,10)
x = SymbolicVar('x', x_value)
y = SymbolicVar('y', y_value)
z1 = x * x + x * y * 5 + 4
z1_diff_x = z1.backward(x).compute()
z1_diff_y = z1.backward(y).compute()

z2 = x ** y
z2_diff_x = z2.backward(x).compute()
z2_diff_y = z2.backward(y).compute()

z3 = (3 * x) / y
z3_diff_x = z3.backward(x).compute()
z3_diff_y = z3.backward(y).compute()

#sympy setup
x = sympy.Symbol('x')
y = sympy.Symbol('y')
z1 = x * x + x * y * 5 + 4
assert sympy.lambdify([x,y], sympy.diff(z1, x))(x_value,y_value) == z1_diff_x
assert sympy.lambdify([x,y], sympy.diff(z1, y))(x_value,y_value) == z1_diff_y

z2 = x ** y
assert sympy.lambdify([x,y], sympy.diff(z2, x))(x_value,y_value) == z2_diff_x
assert sympy.lambdify([x,y], sympy.diff(z2, y))(x_value,y_value) == z2_diff_y

z3 = (3 * x) / y
assert sympy.lambdify([x,y], sympy.diff(z3, x))(x_value,y_value) == z3_diff_x
assert sympy.lambdify([x,y], sympy.diff(z3, y))(x_value,y_value) == z3_diff_y
print("Passed  🎉")


Passed  🎉


In [61]:
#testing exp, cos, sin, log
x_value, y_value = np.random.randint(1,100), np.random.randint(1,10)
x = SymbolicVar('x', x_value)
y = SymbolicVar('y', y_value)
z1 = Exp(x + y)
z1_diff_x = z1.backward(x).compute()
z1_diff_y = z1.backward(y).compute()

z2 = Cos(x - y)
z2_diff_x = z2.backward(x).compute()
z2_diff_y = z2.backward(y).compute()

z3 = Sin(x * y)
z3_diff_x = z3.backward(x).compute()
z3_diff_y = z3.backward(y).compute()

z4 = Log(Sin(x) + Cos(y))
z4_diff_x = z4.backward(x).compute()
z4_diff_y = z4.backward(y).compute()

#sympy setup
x = sympy.Symbol('x')
y = sympy.Symbol('y')
z1 = sympy.exp(x + y)
assert sympy.lambdify([x,y], sympy.diff(z1, x))(x_value,y_value) == z1_diff_x
assert sympy.lambdify([x,y], sympy.diff(z1, y))(x_value,y_value) == z1_diff_y

z2 = sympy.cos(x - y)
assert sympy.lambdify([x,y], sympy.diff(z2, x))(x_value,y_value) == z2_diff_x
assert sympy.lambdify([x,y], sympy.diff(z2, y))(x_value,y_value) == z2_diff_y

z3 = sympy.sin(x * y)
assert sympy.lambdify([x,y], sympy.diff(z3, x))(x_value,y_value) == z3_diff_x
assert sympy.lambdify([x,y], sympy.diff(z3, y))(x_value,y_value) == z3_diff_y

z4 = sympy.log(sympy.sin(x) + sympy.cos(y))
assert sympy.lambdify([x,y], sympy.diff(z4, x))(x_value,y_value) == z4_diff_x
assert sympy.lambdify([x,y], sympy.diff(z4, y))(x_value,y_value) == z4_diff_y
print("Passed  🎉")

Passed  🎉


Almost here 🎉 Two steps left: implement own numerical differentation mechanism & generator of testing functions. Let's start with first one (covers 1.3 task).

**Newton's difference quotient** -- according to [wiki](https://en.wikipedia.org/wiki/Numerical_differentiation), our diff can be represented as finite difference approximations.




In [62]:
def partial_difference_quotient(f, var, i, step):
    """compute the i-th partial difference quotient of function f at var"""
    var_step = [var_j + (step if j == i else 0) for j, var_j in enumerate(var)]

    return (f(var_step) - f(var)) / step #slope

def estimate_gradient(f, var, step=0.00001):
    return [partial_difference_quotient(f, var, i, step) for i, _ in enumerate(var)]

In [63]:
x_value, y_value = np.random.randint(1,100), np.random.randint(1,10)
print(f"x: {x_value}, y: {y_value}")
x = SymbolicVar('x', x_value)
y = SymbolicVar('y', y_value)
z = x * x + x * y * 3 + 1 
print(f"autograd dz/dx = {z.backward(x).compute()}")
print(f"autograd dz/dy = {z.backward(y).compute()}")

def f(v):
    x, y = v
    return x * x + x * y * 3 + 1 

var = [x_value, y_value]
grad = estimate_gradient(f, var, step = 0.00001)
formatted_grad = [ '%.2f' % elem for elem in grad ]
print(f"numerical diff dz/dx ~= {formatted_grad[0]} (full form: {grad[0]})")
print(f"numerical diff dz/dy ~= {formatted_grad[1]} (full form: {grad[1]})")

x: 97, y: 7
autograd dz/dx = 215.0
autograd dz/dy = 291.0
numerical diff dz/dx ~= 215.00 (full form: 215.00001003005306)
numerical diff dz/dy ~= 291.00 (full form: 290.9999999246793)


In [64]:
x_value, y_value = np.random.randint(1,100), np.random.randint(1,10)
print(f"x: {x_value}, y: {y_value}")
x = SymbolicVar('x', x_value)
y = SymbolicVar('y', y_value)
z = Abs(x-y)
print(f"autograd dz/dx = {z.backward(x).compute()}")
print(f"autograd dz/dy = {z.backward(y).compute()}")

def f(v):
    x, y = v
    return abs(x - y)

var = [x_value, y_value]
grad = estimate_gradient(f, var, step = 0.00001)
formatted_grad = [ '%.2f' % elem for elem in grad ]
print(f"numerical diff dz/dx ~= {formatted_grad[0]} (full form: {grad[0]})")
print(f"numerical diff dz/dy ~= {formatted_grad[1]} (full form: {grad[1]})")

x: 18, y: 6
autograd dz/dx = 1
autograd dz/dy = 1
numerical diff dz/dx ~= 1.00 (full form: 0.9999999999621422)
numerical diff dz/dy ~= -1.00 (full form: -0.9999999999621422)


As you can see, pretty close to our auto-diff's results! And last but now least step: let's create toy functions' generator.

It would be nice to test if result of auto-diff of generated function is the same as result of numerical diff of generated function, so we will keep this idea in mind and write method which generates mathematically the same function but in two different forms - as an input for auto-diff and as an input for numerical one.

Additionally, let's fully cover task 1.5 by allow to generate multivariable functions


In [65]:
import ast
import math
from typing import Callable

In [66]:
def _arg_name(i: int):
    return f"x{i}"

def _random_expr(args_amount: int, recursion_limit: int):
    recursion_limit -= 1

    if recursion_limit == 1:
        unary_node_without_recursive_calls_type = random.choice([
            'const',
            'var',
        ])
        if unary_node_without_recursive_calls_type == 'const':
            value = random.randint(-10, 10)
            return [
                ast.Call(ast.Name(id='SymbolicConst', ctx=ast.Load()), args=[ast.Constant(value)], keywords=[]),
                ast.Constant(value)
            ] 
        elif unary_node_without_recursive_calls_type == 'var':
            name = _arg_name(random.randint(0,args_amount-1))
            return [ast.Name(id=name, ctx=ast.Load()), ast.Name(id=name, ctx=ast.Load())]


    # choose binary or unary node
    if random.random() < 0.4:
        # expression will be bin op
        left = _random_expr(args_amount,recursion_limit)
        right = _random_expr(args_amount, recursion_limit)
        op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div(), ast.Pow()])
        return [
            ast.BinOp(
                left=left[0],
                op=op,
                right=right[0]
            ),
            ast.BinOp(
                left=left[1],
                op=op,
                right=right[1]
            )
        ]
    else:
        # expression will be single node
        unary_node_type = random.choice([
            'const',
            'var',
            'abs',
            'cos',
            'sin',
            'exp',
            'log',
        ])
        if unary_node_type == 'const':
            value = random.randint(-10, 10)
            return [
                ast.Call(ast.Name(id='SymbolicConst', ctx=ast.Load()), args=[ast.Constant(value)], keywords=[]),
                ast.Constant(value)
            ]
        if unary_node_type == 'var':
            name = _arg_name(random.randint(0,args_amount-1))
            return [ast.Name(id=name, ctx=ast.Load()), ast.Name(id=name, ctx=ast.Load())]
        elif unary_node_type == 'abs':
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(ast.Name(id='Abs', ctx=ast.Load()), args=[expr[0]], keywords=[]),
                ast.Call(ast.Name(id='abs', ctx=ast.Load()), args=[expr[1]], keywords=[])
            ]
        elif unary_node_type == 'cos':
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(ast.Name(id='Cos', ctx=ast.Load()), args=[expr[0]], keywords=[]),
                ast.Call(ast.Name(id='cos', ctx=ast.Load()), args=[expr[1]], keywords=[])
            ]
        elif unary_node_type == 'sin':
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(ast.Name(id='Sin', ctx=ast.Load()), args=[expr[0]], keywords=[]),
                ast.Call(ast.Name(id='sin', ctx=ast.Load()), args=[expr[1]], keywords=[])
            ]
        elif unary_node_type == 'log':
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(ast.Name(id='Log', ctx=ast.Load()), args=[expr[0]], keywords=[]),
                ast.Call(ast.Name(id='log', ctx=ast.Load()), args=[expr[1]], keywords=[])
            ]
        elif unary_node_type == 'exp':
            expr = _random_expr(args_amount, recursion_limit)
            return [
                ast.Call(ast.Name(id='Exp', ctx=ast.Load()), args=[expr[0]], keywords=[]),
                ast.Call(ast.Name(id='exp', ctx=ast.Load()), args=[expr[1]], keywords=[])
            ]


def generate_function(args_amount: int = 1, recursion_limit: int = 10, verbose=True) -> Callable[[float], float]:
    body = _random_expr(args_amount, recursion_limit)
    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[
                    ast.arg(arg=_arg_name(i))
                    for i in range(args_amount)
                ],
                posonlyargs=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]
            ),
            body=body[0]
        )
    )

    expr_for_num_diff = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[
                    ast.arg(arg=_arg_name(i))
                    for i in range(args_amount)
                ],
                posonlyargs=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]
            ),
            body=body[1]
        )
    )

    ast.fix_missing_locations(expr)
    ast.fix_missing_locations(expr_for_num_diff)
    if verbose:
        print(f"Generated function for auto-diff: {ast.unparse(expr)}")
        print(f"Generated function for numerical diff: {ast.unparse(expr_for_num_diff)}\n")
    compiled_func = compile(expr, filename="", mode="eval")
    compiled_func_for_num_diff = compile(expr_for_num_diff, filename="", mode="eval")
    func = eval(compiled_func, {'Abs': Abs, 'Cos': Cos, 'Sin': Sin, 'Log': Log, 'Exp': Exp, 'SymbolicConst': SymbolicConst})
    func_for_numerical_diff = eval(compiled_func_for_num_diff, {'abs':abs, 'cos': math.cos, 'sin': math.sin, 'log': math.log,'exp': math.exp})
    return func, func_for_numerical_diff

Our toy auto-gen sometimes produces funny things, such as many stacked exponents which causes inf as a result; unfortunately, because we generate functions randomly, this kind of behaviour is necessary evil, just reload cell 🤓 (~~probably it can be fixed somehow at least author doesn't know how~~)

But let's take a look at the result of all things below:

In [67]:
x = SymbolicVar('x', 10)
func, func4num = generate_function(recursion_limit=4)
print(f"Generated function: {func(x)}")
print(f"Generated function at point {x.value}: {func(x).compute():.2f}")
print(f"Generated function derivative: {func(x).backward(x)}")
print(f"Generated function derivative at point {x.value}: {func(x).backward(x).compute():.2f}\n")

def f(v):
    x = v[0]
    return func4num(x)

var = [x.value]
grad = estimate_gradient(f, var, step = 0.00001)
print(f"Generated function numerical derivative at point {x.value}: {grad[0]:.2f}")


Generated function for auto-diff: lambda x0: Log(x0) * Abs(SymbolicConst(-7))
Generated function for numerical diff: lambda x0: log(x0) * abs(-7)

Generated function: (log(x) * (|-7|))
Generated function at point 10: 16.12
Generated function derivative: ((((1 / x) * 1) * (|-7|)) + (log(x) * (|0.0|)))
Generated function derivative at point 10: 0.70

Generated function numerical derivative at point 10: 0.70


In [68]:
x = SymbolicVar('x', 10)
y = SymbolicVar('y', 5)
func, func4num = generate_function(args_amount=2, recursion_limit=5)
print(f"Generated function: {func(x, y)}")
print(f"Generated function at point ({x.value}, {y.value}): {func(x, y).compute():.2f}")
print(f"Generated function derivative by x: {func(x,y).backward(x)}")
print(f"Generated function derivative by y: {func(x,y).backward(y)}")
print(f"Generated function derivative by x at point ({x.value}, {y.value}): {func(x,y).backward(x).compute():.2f}")
print(f"Generated function derivative by y at point ({x.value}, {y.value}): {func(x,y).backward(y).compute():.2f}\n")

def f(v):
    x, y = v
    return func4num(x, y)

var = [x.value, y.value]
grad = estimate_gradient(f, var, step = 0.00001)
formatted_grad = [ '%.2f' % elem for elem in grad ]
print(f"numerical diff dz/dx ~= {formatted_grad[0]} (full form: {grad[0]})")
print(f"numerical diff dz/dy ~= {formatted_grad[1]} (full form: {grad[1]})")

Generated function for auto-diff: lambda x0, x1: Sin(Abs(SymbolicConst(10) * SymbolicConst(-4)))
Generated function for numerical diff: lambda x0, x1: sin(abs(10 * -4))

Generated function: sin((|(10 * -4)|))
Generated function at point (10, 5): 0.75
Generated function derivative by x: (cos((|(10 * -4)|)) * (|((0.0 * -4) + (10 * 0.0))|))
Generated function derivative by y: (cos((|(10 * -4)|)) * (|((0.0 * -4) + (10 * 0.0))|))
Generated function derivative by x at point (10, 5): -0.00
Generated function derivative by y at point (10, 5): -0.00

numerical diff dz/dx ~= 0.00 (full form: 0.0)
numerical diff dz/dy ~= 0.00 (full form: 0.0)


In [69]:
x1 = SymbolicVar('x1', 10)
x2 = SymbolicVar('x2', 5)
x3 = SymbolicVar('x3', -2)
func, func4num = generate_function(args_amount=3, recursion_limit=5)
print(f"Generated function: {func(x1, x2, x3)}")
print(f"Generated function at point ({x1.value}, {x2.value}, {x3.value}): {func(x1, x2, x3).compute():.2f}\n")
print(f"Generated function derivative by x1: {func(x1, x2, x3).backward(x1)}")
print(f"Generated function derivative by x2: {func(x1, x2, x3).backward(x2)}")
print(f"Generated function derivative by x3: {func(x1, x2, x3).backward(x3)}\n")
print(f"Generated function derivative by x1 at point ({x1.value}, {x2.value}, {x3.value}): {func(x1, x2, x3).backward(x1).compute():.2f}")
print(f"Generated function derivative by x2 at point ({x1.value}, {x2.value}, {x3.value}): {func(x1, x2, x3).backward(x2).compute():.2f}")
print(f"Generated function derivative by x3 at point ({x1.value}, {x2.value}, {x3.value}): {func(x1, x2, x3).backward(x3).compute():.2f}\n")

def f(v):
    x1, x2, x3 = v
    return func4num(x1, x2, x3)

var = [x1.value, x2.value, x3.value]
grad = estimate_gradient(f, var, step = 0.00001)
formatted_grad = [ '%.2f' % elem for elem in grad ]
print(f"numerical diff dz/dx1 ~= {formatted_grad[0]} (full form: {grad[0]})")
print(f"numerical diff dz/dx2 ~= {formatted_grad[1]} (full form: {grad[1]})")
print(f"numerical diff dz/dx3 ~= {formatted_grad[2]} (full form: {grad[2]})")

Generated function for auto-diff: lambda x0, x1, x2: SymbolicConst(-4) + (x1 + x2) - Cos(Log(SymbolicConst(8)))
Generated function for numerical diff: lambda x0, x1, x2: -4 + (x1 + x2) - cos(log(8))

Generated function: ((-4 + (x2 + x3)) + (-cos(log(8))))
Generated function at point (10, 5, -2): -0.51

Generated function derivative by x1: ((0.0 + (0 + 0)) + (-((-sin(log(8))) * ((1 / 8) * 0.0))))
Generated function derivative by x2: ((0.0 + (1 + 0)) + (-((-sin(log(8))) * ((1 / 8) * 0.0))))
Generated function derivative by x3: ((0.0 + (0 + 1)) + (-((-sin(log(8))) * ((1 / 8) * 0.0))))

Generated function derivative by x1 at point (10, 5, -2): 0.00
Generated function derivative by x2 at point (10, 5, -2): 1.00
Generated function derivative by x3 at point (10, 5, -2): 1.00

numerical diff dz/dx1 ~= 0.00 (full form: 0.0)
numerical diff dz/dx2 ~= 1.00 (full form: 0.9999999999621422)
numerical diff dz/dx3 ~= 1.00 (full form: 1.0000000000065512)


## Задача с экзамена

In [70]:
class _Helper:
    def __init__(self, node):
        self.node = node

    def __add__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return _Helper(Sum(self.node, Const(other)))
        if isinstance(other, _Helper):
            return _Helper(Sum(self.node, other.node))
        raise TypeError()

    def __radd__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return _Helper(Sum(Const(other), self.node))
        if isinstance(other, _Helper):
            return _Helper(Sum(other.node, self.node))
        raise TypeError()

    def __mul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return _Helper(Mul(self.node, Const(other)))
        if isinstance(other, _Helper):
            return _Helper(Mul(self.node, other.node))
        raise TypeError()

    def __rmul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return _Helper(Mul(Const(other), self.node))
        if isinstance(other, _Helper):
            return _Helper(Mul(Const(other), self.node))
        raise TypeError()

def nodify(f):
    return f(_Helper(Variable('x'))).node

In [71]:
nodify(lambda x: x*2 + 2)

((x * 2) + 2)